# A program to generate statistics on Absolute Radio

The program monitors the weekday show (9am - 6pm) on absolute radio and will:

1. 2. generate statistics on the number of songs played
3. (in future) keep a tally of genres so that we can make some interesting plots in plotly

There are a few things that must be implemented

1. Getting data to plotly
2. Running program only on specific days of the week
3. Genres

In [186]:
'''
Created on 15 Jan 2017

@author: afunn
'''
print("Absolute Radio - Did they repeat?")

Absolute Radio - Did they repeat?


### Import the required libraries

In [187]:
from lxml import html  # for scraping web information
import requests
from threading import Timer  # for timing
from collections import Counter  # for counting list
import time
import plotly
from datetime import datetime
import pandas as pd # for dataframes used for plotly
from random import randint # for random numbers for testing
plotly.tools.set_credentials_file(username='benisme', api_key='clBZNegtB5pqHJZXRnnP')
plotly.__version__

'2.0.8'

### Definitions
#### Create the global variables that are used throughout the program
It is generally not best practice to do this, but because we are using the timer, it is quite a difficult problem to get around and this is an easy solution

#### Other Definitions
We use a number of lists to keep track of what's happening with the songs and counts

In [188]:
# Global variables (Very bad to do this :( but its easy)
global CurrentRepeatList
global CurrentRepeatListLength
CurrentRepeatList = []
CurrentRepeatListLength = 0

### Here we write the code to strip the data from the absolute radio webpage

This is needed at the start of the program so that the starting state of the program is defined.
We place this into three arrays, *artists*, *songs* and *times*

In [189]:
# Get the data from absolute radio

page = requests.get('https://absoluteradio.co.uk/absolute-radio/music/')
tree = html.fromstring(page.content)

# This will create a list of artists
artists = tree.xpath('//p[@class="song-artist"]/text()')
# This will create a list of songs
songs = tree.xpath('//p[@class="song-title"]/a/text()')
# This will create a list of the times of the songs
times = tree.xpath('//div[@class="song-inner"]/time/text()')

We then print these to the console, just so that we know things are working

In [190]:
print(artists)
print(songs)
print(times)

['New Order', 'The Darkness', 'Travis', 'Florence and the Machine', 'Marillion', 'The Lightning Seeds', 'Richard Ashcroft', 'Guns n Roses', 'Kings Of Leon', 'Urge Overkill', 'The Fratellis', 'AC/DC']
['Regret', 'I Believe In A Thing Called Love', 'Writing To Reach You', 'Spectrum (Say My Name)', 'Kayleigh', 'Pure', 'A Song For The Lovers', 'Live And Let Die', 'Sex On Fire', "Girl You'll Be A Woman Soon", 'Chelsea Dagger ', 'Play Ball']
['10.32am', '10.28am', '10.25am', '10.21am', '10.16am', '10.12am', '10.08am', '10.05am', '10.02am', '9.59am', '9.52am', '9.49am']


Here we will change the times in the *ArtistsSongsTimesCount* list to standard format

In [191]:
 
print(times) # Troubleshooting code

def timeconversion_and_committ(times_list):
    x = 0
    for item in times_list:
        time_string = item
        #plus_string = '2009-11-29'
        plus_string = datetime.today().strftime("%Y-%m-%d")
        newformat = '%Y-%m-%d %I.%M%p'
        format1 = '%Y-%m-%d %I:%M %p'
        total_string = plus_string + " " + time_string
        my_date = datetime.strptime(total_string, newformat)
        times_list[x] = my_date.strftime(format1)
        x = x+1
    
    #print(times_list)
    return(times_list)

times = timeconversion_and_committ(times)
print(times)


['10.32am', '10.28am', '10.25am', '10.21am', '10.16am', '10.12am', '10.08am', '10.05am', '10.02am', '9.59am', '9.52am', '9.49am']
['2017-05-16 10:32 AM', '2017-05-16 10:28 AM', '2017-05-16 10:25 AM', '2017-05-16 10:21 AM', '2017-05-16 10:16 AM', '2017-05-16 10:12 AM', '2017-05-16 10:08 AM', '2017-05-16 10:05 AM', '2017-05-16 10:02 AM', '2017-05-16 09:59 AM', '2017-05-16 09:52 AM', '2017-05-16 09:49 AM']


And we define two variable lists based on the length of the number of songs that was returned.

In [192]:
# Main list definitions
ArtistsSongsTimesCount = [artists, songs, times, [0] * len(artists)]

"""# This is just code to assign a value in count - this can be replaced by the duration function ! 
for x in range(0,(len(ArtistsSongsTimesCount[3]))):
    #ArtistsSongsTimesCount[3][x] = x + 1
    if x>8:
        ArtistsSongsTimesCount[3][x] = '0:06:00'
    elif x>5:
        ArtistsSongsTimesCount[3][x] = '0:03:00'
    else:
        ArtistsSongsTimesCount[3][x] = '0:04:00'
    
print(ArtistsSongsTimesCount[3])"""


def calculate_durations(Local_ArtistsSongsTimesCount):
    temporary_storage = Local_ArtistsSongsTimesCount[2][(len(Local_ArtistsSongsTimesCount[0]) - 1)]
    OutputFinal = []
    for i in range(0,(len(Local_ArtistsSongsTimesCount[0]) - 1)):
        format1 = '%Y-%m-%d %I:%M %p'
        format2 = '%M'
        my_date1 = datetime.strptime(Local_ArtistsSongsTimesCount[2][i], format1)
        my_date2 = datetime.strptime(Local_ArtistsSongsTimesCount[2][i+1], format1)
        ReadyForOutput = (my_date1 - my_date2).seconds
        OutputFinal.append(ReadyForOutput)
    
    if isinstance(temporary_storage, str):
        OutputFinal.append(195)
    else:
        OutputFinal.append(temporary_storage)
    return OutputFinal

ArtistsSongsTimesCount[3]= calculate_durations(ArtistsSongsTimesCount)
print(ArtistsSongsTimesCount[3])




[240, 180, 240, 300, 240, 240, 180, 180, 180, 420, 180, 195]


Here we write to a file at the start of the program all of the songs which have been played so far. This will become our master file which we will use to generated statistics.

In [193]:
OutputFinal = []

f = open('Today_Songlist', 'w')

f.write('Artists;Songs;Times;Genres\n')

for i in range((len(ArtistsSongsTimesCount[0]) - 1),0,-1):
    ReadyForOutput = ""
    for list in ArtistsSongsTimesCount:
        ReadyForOutput = ReadyForOutput + str(list[i]) + ';'
    ReadyForOutput = ReadyForOutput + '\n'
    OutputFinal.append(ReadyForOutput)

for item in OutputFinal:
    f.write(item)
    
f.close()

Testing that the file reads

In [194]:
f = open('Today_Songlist', 'r')
hello = f.read()
f.close()

print(hello)


Artists;Songs;Times;Genres
AC/DC;Play Ball;2017-05-16 09:49 AM;195;
The Fratellis;Chelsea Dagger ;2017-05-16 09:52 AM;180;
Urge Overkill;Girl You'll Be A Woman Soon;2017-05-16 09:59 AM;420;
Kings Of Leon;Sex On Fire;2017-05-16 10:02 AM;180;
Guns n Roses;Live And Let Die;2017-05-16 10:05 AM;180;
Richard Ashcroft;A Song For The Lovers;2017-05-16 10:08 AM;180;
The Lightning Seeds;Pure;2017-05-16 10:12 AM;240;
Marillion;Kayleigh;2017-05-16 10:16 AM;240;
Florence and the Machine;Spectrum (Say My Name);2017-05-16 10:21 AM;300;
Travis;Writing To Reach You;2017-05-16 10:25 AM;240;
The Darkness;I Believe In A Thing Called Love;2017-05-16 10:28 AM;180;



And then we print some stuff into the console to check things are working

In [195]:
print(len(artists))
print(ArtistsSongsTimesCount)

12
[['New Order', 'The Darkness', 'Travis', 'Florence and the Machine', 'Marillion', 'The Lightning Seeds', 'Richard Ashcroft', 'Guns n Roses', 'Kings Of Leon', 'Urge Overkill', 'The Fratellis', 'AC/DC'], ['Regret', 'I Believe In A Thing Called Love', 'Writing To Reach You', 'Spectrum (Say My Name)', 'Kayleigh', 'Pure', 'A Song For The Lovers', 'Live And Let Die', 'Sex On Fire', "Girl You'll Be A Woman Soon", 'Chelsea Dagger ', 'Play Ball'], ['2017-05-16 10:32 AM', '2017-05-16 10:28 AM', '2017-05-16 10:25 AM', '2017-05-16 10:21 AM', '2017-05-16 10:16 AM', '2017-05-16 10:12 AM', '2017-05-16 10:08 AM', '2017-05-16 10:05 AM', '2017-05-16 10:02 AM', '2017-05-16 09:59 AM', '2017-05-16 09:52 AM', '2017-05-16 09:49 AM'], [240, 180, 240, 300, 240, 240, 180, 180, 180, 420, 180, 195]]


## Plotly
Main Dashboard section

In [196]:
def generate_plotly_dashboard():
    import plotly.dashboard_objs as dashboard
    import plotly.plotly as py

    import IPython.display
    from IPython.display import Image

    my_dboard = dashboard.Dashboard()
    
    text_for_box2 = "### A dashboard displaying some basic Absolute Radio Satistics by using web scraping: \
    \n\n\n #### *The scatter plot* \
    \n\n 1. Shows duration of songs \
    \n 2. Shows title of tracks in order of play \
    \n\n#### *The text element to the left* \
    \n\n 1. Shows the start date/time of the program \
    \n 2. Will let user know if any repeats have taken place"
    
    print(str(datetime.today()))
    format1 = '%Y-%m-%d %I:%M %p'
    my_date = datetime.today()
    my_date = my_date.strftime(format1)
    
    
    box_1 = {
    'type': 'box',
    'boxType': 'plot',
    'fileId': 'benisme:0',
    'shareKey': None,
    'title': 'Timeline of Songs'
    }
    
    box_2 =  {
    'type': 'box',
    'boxType': 'text',
    'text': text_for_box2,
    'title': 'Information'
    }
    
    box_3 =  {
    'type': 'box',
    'boxType': 'text',
    'text': my_date,
    'title': 'Date / Song Count'
    }
    
    my_dboard.insert(box_1)
    my_dboard.insert(box_2, 'below', 1)
    my_dboard.insert(box_3, 'left', 2)
    
    my_dboard['settings']['title'] = 'Absolute Radio Statistics'
    
    my_dboard['settings']['logoUrl'] = 'https://cdn.pixabay.com/photo/2016/01/03/11/32/wireless-signal-1119306_960_720.png'
    
    py.dashboard_ops.upload(my_dboard, 'Absolute Radio Statistics')
    

In [197]:
#my_dboard.get_preview()

### Plotly graphs no. 1 for dashboard

timeline for the dashboard

In [198]:
def generate_plotly_graph1():
    
    import plotly.plotly as py
    import plotly.graph_objs as go

    import pandas as pd
    import numpy as np

    headers = ['Artists', 'Songs', 'DateTime', 'Genres']
    dtypes = {'Artists': 'str', 'Songs': 'str', 'DateTime': 'str', 'Genres': 'str'}
    parse_dates = ['DateTime']

    df = pd.read_csv('Today_Songlist', sep=';', header=None, names=headers, dtype=dtypes, parse_dates=parse_dates)

    #df = pd.read_csv('Today_Songlist', sep=';')
    #df = pd.read_csv('Today_Songlist',sep=';', parse_dates=['Times'])


    df.drop(df.index[[0]], inplace=True)
    print(df)
    


    trace_high = go.Scatter(
                    x=df.DateTime,
                    y=df['Genres'],
                    mode='markers',marker=dict(
                        size='16',
                        color = np.random.randn(500)),
                    text=df['Songs'],
                    name = "Genres",
                    line = dict(color = '#17BECF'),
                    opacity = 0.8)

    data = [trace_high]

    layout = dict(
        title = "Absolute Radio - Recent Song Durations",yaxis=dict(title='Song durations (seconds)'))
        #xaxis = dict(range = ['2017-05-12','2017-05-15'])
    

    fig = dict(data=data, layout=layout)
    py.iplot(fig, filename = "Manually Set Range")



### The master function

Here we define the master function which will be called by the timer, this will
1. Call the *Retrieve_TimesArtistsSongs()* function 
2. If the *Retrieve_TimesArtistsSongs()* returns true, indicating a new song has been played, the master function will
  * Check whether it is a repeat: It does this by running *UpdateCount()*
  * *Update Count* Returns the current list of repeated songs, and the number of songs played
  * If the length of the current list of repeated songs has changed from the last loop, then actions are generated
  
3. Return values back to the timer

In [199]:
# start of master function definition
def update_song_list_count_and_email():
    global CurrentRepeatList
    global CurrentRepeatListLength

    NewSongBoolean = False
    RepeatBoolean = False
    TotalSongs = 0

    NewSongBoolean = Retrieve_TimesArtistsSongs()
    print("")
    print("NewSongBoolean = ", NewSongBoolean)
    # change this to a case statement later on
    if NewSongBoolean == True:
        print("Updating List")
        update_songs_file()
        generate_plotly_graph1()
        

    else:
        print("Not updating this time around")

    print("CurrentRepeatList = ", CurrentRepeatList)
    print("len(CurrentRepeatList) = ", len(CurrentRepeatList))

    CurrentRepeatListLength = len(CurrentRepeatList)

    # If the update count returns a bigger value then do something

    print("")
    print("ArtistsSongsTimesCount = ", ArtistsSongsTimesCount)
    print("")
   
    # Function check to see if count is > EmailNotificaiton list

    return CurrentRepeatListLength, TotalSongs

## The *Retrieve_TimesArtistsSongs* function

What this function is doing is:
1. Getting the current list of Artists, songs, times from the absolute radio website
2. Comparing the songs list to the existing songs list, and if there is a difference
  1. It inserts the new song into the existing array
  2. It returns the value 'true'

In [200]:
def Retrieve_TimesArtistsSongs():
    page = requests.get('https://absoluteradio.co.uk/absolute-radio/music/')
    tree = html.fromstring(page.content)
    # This will create a list of artists
    artists_B = tree.xpath('//p[@class="song-artist"]/text()')
    # This will create a list of songs
    songs_B = tree.xpath('//p[@class="song-title"]/a/text()')
    # This will create a list of the times of the songs
    times_B = tree.xpath('//div[@class="song-inner"]/time/text()')
    
    times_B = timeconversion_and_committ(times_B)
    
    print(times_B)

    if songs_B[0] != ArtistsSongsTimesCount[1][0]: # i.e. compare the newly created songs array to the existing songs array
        ArtistsSongsTimesCount[0].insert(0, artists_B[0])
        ArtistsSongsTimesCount[1].insert(0, songs_B[0])
        ArtistsSongsTimesCount[2].insert(0, times_B[0])
        #ArtistsSongsTimesCount[3].insert(0, randint(0,9))
        
        ArtistsSongsTimesCount[3]= calculate_durations(ArtistsSongsTimesCount)
        
        Length = (len(ArtistsSongsTimesCount[0]) - 1)
        for x in ArtistsSongsTimesCount:
            del x[Length]
        
        return True

    else:
        return False

### The update the songs record file on addition of new song function
    

In [201]:
def update_songs_file():
    OutputFinal = []
    ReadyForOutput = ""
    
    f = open('Today_Songlist', 'a')

    for list in ArtistsSongsTimesCount:
        ReadyForOutput = ReadyForOutput + str(list[0]) + ';'
    ReadyForOutput = ReadyForOutput + '\n'
    OutputFinal.append(ReadyForOutput)

    for item in OutputFinal:
        f.write(item)

    f.close()

## Start of main program

In [ ]:
generate_plotly_graph1()
generate_plotly_dashboard()

Length = len(ArtistsSongsTimesCount[0])
for x in ArtistsSongsTimesCount:
    for y in range(3, Length):
        del x[3]

                     Artists                             Songs  \
1                      AC/DC                         Play Ball   
2              The Fratellis                   Chelsea Dagger    
3              Urge Overkill       Girl You'll Be A Woman Soon   
4              Kings Of Leon                       Sex On Fire   
5               Guns n Roses                  Live And Let Die   
6           Richard Ashcroft             A Song For The Lovers   
7        The Lightning Seeds                              Pure   
8                  Marillion                          Kayleigh   
9   Florence and the Machine            Spectrum (Say My Name)   
10                    Travis              Writing To Reach You   
11              The Darkness  I Believe In A Thing Called Love   

               DateTime Genres  
1   2017-05-16 09:49 AM    195  
2   2017-05-16 09:52 AM    180  
3   2017-05-16 09:59 AM    420  
4   2017-05-16 10:02 AM    180  
5   2017-05-16 10:05 AM    180  
6   2017-

## Timer Function

In [ ]:
# Creates a timer function running for 30s, and running function get songs

starttime = time.time()
while True:
    CurrentRepeatList2, TotalSongs2 = update_song_list_count_and_email()
    # Code needed here to
    # pass back whether there has been a repeat and the total number of songs.
    # also check if need to re-tweet at someone
    time.sleep(30.0 - ((time.time() - starttime) % 30.0))

['2017-05-16 10:32 AM', '2017-05-16 10:28 AM', '2017-05-16 10:25 AM', '2017-05-16 10:21 AM', '2017-05-16 10:16 AM', '2017-05-16 10:12 AM', '2017-05-16 10:08 AM', '2017-05-16 10:05 AM', '2017-05-16 10:02 AM', '2017-05-16 09:59 AM', '2017-05-16 09:52 AM', '2017-05-16 09:49 AM']

NewSongBoolean =  False
Not updating this time around
CurrentRepeatList =  []
len(CurrentRepeatList) =  0

ArtistsSongsTimesCount =  [['New Order', 'The Darkness', 'Travis'], ['Regret', 'I Believe In A Thing Called Love', 'Writing To Reach You'], ['2017-05-16 10:32 AM', '2017-05-16 10:28 AM', '2017-05-16 10:25 AM'], [240, 180, 240]]

['2017-05-16 10:32 AM', '2017-05-16 10:28 AM', '2017-05-16 10:25 AM', '2017-05-16 10:21 AM', '2017-05-16 10:16 AM', '2017-05-16 10:12 AM', '2017-05-16 10:08 AM', '2017-05-16 10:05 AM', '2017-05-16 10:02 AM', '2017-05-16 09:59 AM', '2017-05-16 09:52 AM', '2017-05-16 09:49 AM']

NewSongBoolean =  False
Not updating this time around
CurrentRepeatList =  []
len(CurrentRepeatList) =  0

A

['2017-05-16 10:40 AM', '2017-05-16 10:32 AM', '2017-05-16 10:28 AM', '2017-05-16 10:25 AM', '2017-05-16 10:21 AM', '2017-05-16 10:16 AM', '2017-05-16 10:12 AM', '2017-05-16 10:08 AM', '2017-05-16 10:05 AM', '2017-05-16 10:02 AM', '2017-05-16 09:59 AM', '2017-05-16 09:52 AM', '2017-05-16 09:49 AM']

NewSongBoolean =  True
Updating List
                     Artists                             Songs  \
1                      AC/DC                         Play Ball   
2              The Fratellis                   Chelsea Dagger    
3              Urge Overkill       Girl You'll Be A Woman Soon   
4              Kings Of Leon                       Sex On Fire   
5               Guns n Roses                  Live And Let Die   
6           Richard Ashcroft             A Song For The Lovers   
7        The Lightning Seeds                              Pure   
8                  Marillion                          Kayleigh   
9   Florence and the Machine            Spectrum (Say My Name)   
10

Ideas for next steps

In [ ]:
"""    
At end of day (1) tweet about the number of songs and repeats and (2) write day's output to file'

I can have 2 seperate functions (1) that tweets to twitter (2) one that checks twitter



Settings file: 

days of the week to run

start time

stop time


Generated file:

How many full days monitored

How many repeats
"""